In [15]:
# !pip install qiskit qiskit-aer shortuuid

In [16]:
#!/usr/bin/env python
# coding: utf-8

import numpy as np
from qiskit import *
import time
import multiprocessing as mp
import tensorflow as tf
from multiprocessing import cpu_count

In [17]:
# Importing Qiskit
from qiskit.qasm2 import dumps
from qiskit_aer import Aer
import matplotlib.pyplot as plt

import hashlib
import base64
from numpy import log as ln
import shortuuid
import random
from random import randrange

import os
import sys
import logging
import pickle
import json

In [18]:
np.set_printoptions(precision=4)

# Set the multiprocessing start method to 'spawn' to avoid issues with forking and GPU/TPU.
# mp.set_start_method('spawn', force=True)

# Set this to True if you want to use hardware acceleration (GPU/TPU)
accelerate = True

# sys.path.insert(1, '/kaggle/input/checkpoint-2-5-files')

In [19]:
%run /kaggle/input/checkpoint-3-5/routine_error_correction.ipynb
%run /kaggle/input/checkpoint-3-5/routine_helper1.ipynb
%run /kaggle/input/checkpoint-3-5/routine_key.ipynb
%run /kaggle/input/checkpoint-3-5/routine_plot_cascade1.ipynb
# 
# %run /kaggle/input/checkpoint-3-5/routine_error_correction.ipynb

In [20]:
# Function to select the appropriate hardware accelerator (GPU, TPU, or CPU)
def select_device():
    if accelerate:
        try:
            # Check for TPU availability
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            print('Running on TPU:', tpu.cluster_spec().as_dict())
            tf.config.experimental_connect_to_cluster(tpu)
            tf.tpu.experimental.initialize_tpu_system(tpu)
            strategy = tf.distribute.TPUStrategy(tpu)
            return strategy
        except ValueError:
            # If TPU is not available, check for GPU
            gpus = tf.config.list_physical_devices('GPU')
            if gpus:
                print("Running on GPU")
                return None  # TensorFlow will automatically use available GPU
            else:
                print("Running on CPU (NumPy)")
                return None
    else:
        print("Accelerate is False, running on CPU (NumPy).")
        return None

# Select device (GPU/TPU/CPU)
strategy = select_device()

Running on TPU: {}
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


I0000 00:00:1728854703.398697      13 service.cc:145] XLA service 0x570ea366e060 initialized for platform TPU (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728854703.398750      13 service.cc:153]   StreamExecutor device (0): TPU, 2a886c8
I0000 00:00:1728854703.398755      13 service.cc:153]   StreamExecutor device (1): TPU, 2a886c8
I0000 00:00:1728854703.398758      13 service.cc:153]   StreamExecutor device (2): TPU, 2a886c8
I0000 00:00:1728854703.398760      13 service.cc:153]   StreamExecutor device (3): TPU, 2a886c8
I0000 00:00:1728854703.398763      13 service.cc:153]   StreamExecutor device (4): TPU, 2a886c8
I0000 00:00:1728854703.398765      13 service.cc:153]   StreamExecutor device (5): TPU, 2a886c8
I0000 00:00:1728854703.398768      13 service.cc:153]   StreamExecutor device (6): TPU, 2a886c8
I0000 00:00:1728854703.398771      13 service.cc:153]   StreamExecutor device (7): TPU, 2a886c8


INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:0, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:1, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:2, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:3, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:4, TPU, 0, 0)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:5, TPU, 0, 0)
I

In [21]:
help(Key)

Help on class Key in module __main__:

class Key(AliceBob)
 |  Key(index, key_length=None, snr_db=None, *, Eve=None, signal_power=0.15, memory_efficient=True)
 |  
 |  Method resolution order:
 |      Key
 |      AliceBob
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  NoisyChannel(self)
 |      Simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
 |      occur considering SNR.
 |      
 |      Parameters:
 |      - errors: A list to keep track of the number of bit-flip (X) and phase-flip (Z) errors.
 |      - snr_db: Signal-to-Noise Ratio in dB. This affects the probability of errors.
 |      
 |      Returns:
 |      - errors: Updated list with the number of X and Z errors introduced.
 |  
 |  __init__(self, index, key_length=None, snr_db=None, *, Eve=None, signal_power=0.15, memory_efficient=True)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  amplify_privacy(self)
 |  
 |  append_reserves(self, reserve_

In [22]:
# Progress tracking (simple tracker)
def track(batch_idx, num_batches):
    progress = (batch_idx + 1)/num_batches * 100
    print(f"\rProgress: {progress:.2f}%", end = '')

In [23]:
# Generate, encode(in states), noise, measurement
class Key(AliceBob):
        
    log = ' Class created'
    num_keys = 0
    error_prob = 0    # Equivalent to BER

    memory_efficient = 1
    if not memory_efficient:
        kept = np.array([], int)
        discarded = np.array([], int)
        BitsReservoir = [np.array([], dtype='<U11'), np.array([], dtype='<U11')]  # Use a sufficiently large dtype
        BasesReservoir = np.array([], dtype='<U1')  # Assuming bases are single-character strings ('X', 'Z')
        Reservoir = {}

    
    def __init__(self, index, key_length = None, snr_db = None, *, Eve = None, signal_power = 0.15, memory_efficient = True):
        # self.original_key = np.random.randint(0, 2, key_length)  # Random key

        if key_length is None: key_length = 1024
        if snr_db is None: snr_db = 15
        if memory_efficient is None: memory_efficient = False

        Key.memory_efficient = memory_efficient

        self.classical_ber = 0
        self.meta_qber = 0
        self.spot_qber = 0
        self.corrected_qber = 0
        
        self.key_length = key_length    # Desired key length. NOT to be confused with the actual key length
        self.snr_db = snr_db
        self.signal_power = signal_power
        self.qber = 0.    # Before CE. All the errors are due to quantum disturbances
        self.ber = 0.
        self.meta_qber = 0. 
        self.index = index
        
        self.sift_indices = None
        self.final_key = np.array([], dtype='<U11')   # The key object itself can serve as a final key
        self.qc = [QuantumCircuit(1, 1) for _ in range(self.key_length)]    # Initializing the circuit
        self.remarks = 'Initialized'
        self.errors_induced = np.array([0, 0])    # [bit-flip, phase-flip]
        self.status = ''
        
        self.Alice = AliceBob("Alice")
        self.Bob = AliceBob("Bob")
        if Eve is not None: self.Eve = AliceBob("Eve")

        # The length that's needed to create a key of length key_length
        self.redundant_len = int(self.key_length + np.ceil(np.log2(self.key_length)) + 1)
### bits, bases = str, str

    def describe(self):
        # print(f' {self.original_key = }, type: {type(self.original_key)}, {len(self.original_key) = }')
        print(f' {self.qber = },\t type: {type(self.qber)}')
        print(f' {self.ber = },\t type: {type(self.ber)}')
    
        print(f' Desired Key Length: {self.key_length},\t type: {type(self.key_length)}')
        print(f' {self.snr_db = },\t type: {type(self.snr_db)}')
        print(f' {self.signal_power = },\t type: {type(self.signal_power)}')
        print(f' {self.index = },\t type: {type(self.index)}')
        
        print(f' {self.sift_indices = },\t type: {type(self.sift_indices)},\t {sum(self.sift_indices) = }')
        print(f' {self.final_key = },\t type: {type(self.final_key)},\t {len(self.final_key) = }')
        print(f' self.qc : \t type of elements: {type(self.qc[0])},\t {len(self.qc) = }')
        print(f' {self.remarks = },\t type: {type(self.remarks)},\t {len(self.remarks) = }')
        print(f' {self.status = },\t type: {type(self.status)},\t {len(self.status) = }')
        
        print(f' {self.errors_induced = },\t type: {type(self.errors_induced)},\t {len(self.errors_induced) = }')

        print(f' {self.Alice = },\t type: {type(self.Alice)}')
        print(f' {self.Bob = },\t type: {type(self.Bob)}')
        print(f' {self.Eve = },\t type: {type(self.Eve)}')

        print(f' {self.redundant_len = },\t type: {type(self.redundant_len)}')

    
    def generate_random_bits(self, name):
        """This function generates a random array of bits(0/1) of size = key length"""
        obj = self.Alice if name == "Alice" else self.Eve if name == "Eve" else ''
        # for _ in range(self.key_length):
            # rand_bit = random.randint(0, 1)     # = np.random.randint(0, 2)Flip Coin
            # obj.bit_string = np.concatenate((obj.bit_string, [rand_bit]))
        bit_string = np.random.randint(0, 2, self.key_length)
        obj.bit_string = np.array(bit_string).astype(str)
        self.remarks += f'\n {name}: Random Bits generated'
### bits, bases = str, str
    
    def generate_random_bases(self, name):
        """This function selects a random basis for each bit"""
        obj = self.Alice if name == "Alice" else self.Bob if name == "Bob" else self.Eve if name == "Eve" else ''
        # for _ in range(self.key_length):
        #     randBasis = random.randint(0, 1)     # Flip Coin
        #     obj.bases = np.concatenate((obj.bases, ["X"] if randBasis else ["Z"]))
        obj.bases = np.random.choice(['X', 'Z'], self.key_length)
        self.remarks += f'\n {name}: Random Bases generated'
### bits, bases = str, str
        
    def encode_states(self, name):
        """This function encodes each bit into the given basis."""      
        obj = self.Alice if name == "Alice" else self.Eve if name == "Eve" else ''
        for i in range(self.key_length):
            # Possible Cases
            if obj.bit_string[i] == "1": self.qc[i].x(0)
            if obj.bases[i] == 'X': self.qc[i].h(0)
        
        self.remarks += f'\n {name}: States Encoded'

        
    def NoisyChannel(self):
        ''' 
        Simulates a noisy quantum channel where Pauli errors (X - bit flip; Z - phase flip)
        occur considering SNR.
        
        Parameters:
        - errors: A list to keep track of the number of bit-flip (X) and phase-flip (Z) errors.
        - snr_db: Signal-to-Noise Ratio in dB. This affects the probability of errors.
        
        Returns:
        - errors: Updated list with the number of X and Z errors introduced.
        '''
        # Calculate BER
        # Key.error_prob = snr2ber(self.snr_db)
        # print(f'BER = {Key.error_prob}')
        self.errors_induced[0] = 0
        self.errors_induced[1] = 0
               
        # Simulate AWGN effect by modifying measurement outcomes based on noise
        for i in range(self.key_length):
            if random.random() < Key.error_prob:
                self.qc[i].x(0)  # Simulate a bit-flip error due to noise
                self.errors_induced[0] += 1
                
            if random.random() < Key.error_prob:
                self.qc[i].z(0)  # Simulate a phase-flip error due to noise
                self.errors_induced[1] += 1

        self.remarks += '\n Key Transmitted'    # \n "bases" AND "bit_string" RESETTED'


    def measure(self, name):
        """This function measures each qubit in the corresponding basis chosen for it."""
        obj = self.Bob if name == "Bob" else self.Eve if name == "Eve" else ''

        for i in range(self.key_length):
            if obj.bases[i] == "X": 
                self.qc[i].h(0)

            self.qc[i].measure(0, 0)

            # Execute on Simulator
            simulator = Aer.get_backend('qasm_simulator')
            transpiled_circuit = transpile(self.qc[i], simulator)
            result = simulator.run(transpiled_circuit, shots = 1).result()
            counts = result.get_counts()
            measured_bit = max(counts, key = counts.get)     # Max doesn't matter for simulator since there is only one shot.
    
            obj.bit_string = np.concatenate((obj.bit_string, [measured_bit]))
        
        self.remarks += f'\n {name}: States Measured'
### bits, bases = str, str
    
    def sift(self):
        """This function sifts through the bases of both Alice and Bob and checks for the ones that match"""
        self.sift_indices = (self.Bob.bases == self.Alice.bases)    # BROADCAST = self.Alice.bases
        
        self.Bob.bases = self.Bob.bases[self.sift_indices]
        self.Alice.bases = self.Alice.bases[self.sift_indices]

        self.Bob.bit_string = np.array(self.Bob.bit_string)[self.sift_indices]
        self.Alice.bit_string = np.array(self.Alice.bit_string)[self.sift_indices]
    
        # self.Bob.bit_string = ''.join(self.Bob.bit_string)
        self.remarks += f'\n Key Sifted. Sift length = {sum(self.sift_indices)}'
### bits, bases = Array, str

    def spot(self):
        sample = len(self.Alice.bit_string)//3    # len(self.Alice.bit_string) >= 3
        errors_detected = 0
        print('\n Spotting(index, (alice, bob)): ', end = '  ')
        for _ in range(sample):
            bit_index = random.randrange(len(self.Alice.bit_string))
            # print(self.Alice.bit_string[bit_index], self.Bob.bit_string[bit_index], end = '   ')
            
            self.Alice.bit_string = list(self.Alice.bit_string)  # Convert to a list if it's a numpy array
            self.Bob.bit_string = list(self.Bob.bit_string)

            print(f"{bit_index}, ({self.Alice.bit_string[bit_index]}, {self.Bob.bit_string[bit_index]})", end = '   ')
            if self.Alice.bit_string[bit_index] != self.Bob.bit_string[bit_index]:  errors_detected += 1    # calculating errors
            # Remove the bit at `bit_index`
            self.Alice.bit_string = self.Alice.bit_string[:bit_index] + self.Alice.bit_string[bit_index+1:]
            self.Bob.bit_string = self.Bob.bit_string[:bit_index] + self.Bob.bit_string[bit_index+1:]
            
            # Convert back to the numpy array (if needed) after manipulation
            self.Alice.bit_string = np.array(self.Alice.bit_string)
            self.Bob.bit_string = np.array(self.Bob.bit_string)
            
        
        # order = np.ceil(np.log2(len(self.Alice.bit_string))).astype(int)
        order = Order(self.Alice.bit_string)
        
        self.qber = errors_detected/sample    # calculating QBER
        self.qber = round(self.qber, 2)    # saving the answer to two decimal places
        
        print("\nQBER value = ", self.qber)
        # print("alices secret key =", self.Alice.bit_string)
        # print("Bob secret key  =  ", self.Bob.bit_string)

        return errors_detected


    def calculate_meta_qber(self):    # Observer: This data is available only because we are observing from top. Won't be available to users.
        """This function calculates the QBER before applying channel encoding. Can not be used in actual implementation. Only for simulations"""
        meta_qber = np.sum(self.Alice.bit_string != self.Bob.bit_string)/self.Bob.current_length()
        self.meta_qber = meta_qber
        return meta_qber


    def append_reserves(self, reserve_i):

        alice_bit_string = np.array(self.Alice.bit_string[reserve_i:], dtype='<U11')  # Adjust dtype to match		
        bob_bit_string = np.array(self.Bob.bit_string[reserve_i:], dtype='<U11')  # Adjust dtype to match

        if not Key.memory_efficient:
            Key.Reservoir.update({self.index: (self.Bob.bases, self.Bob.bit_string, self.Alice.bit_string)})
            Key.BitsReservoir[0] = np.concatenate((Key.BitsReservoir[0], alice_bit_string))
            Key.BitsReservoir[1] = np.concatenate((Key.BitsReservoir[1], bob_bit_string))
            Key.BasesReservoir = np.concatenate((Key.BasesReservoir, self.Bob.bases[reserve_i:]))

        self.Alice.bit_string = self.Alice.bit_string[:reserve_i]
        self.Bob.bit_string = self.Bob.bit_string[:reserve_i]
        self.Alice.bases = self.Alice.bases[:reserve_i]
        self.Bob.bases = self.Bob.bases[:reserve_i]
# Bits/Bases trimmed

    def fill_block(self, order):
        fill_length = int(2**order - order - 1) - sum(self.sift_indices)
        filler = np.zeros(fill_length)
        return filler
        

    def correct(self, ECC = 'hamming'):
        if ECC == 'hamming': self.correct_hamming()
        elif ECC == 'cascade': self.correct_cascade()


    def correct_cascade(self):
        error, (kcorrA, kcorrB) = biconf(self.qber, self.Alice.bit_string, self.Bob.bit_string)
        self.Alice.bit_string, self.Bob.bit_string = kcorrA, kcorrB

        
    def correct_hamming(self):
        channel_code = HammingCode()

        self.Alice.bit_string[:] = np.array([int(bit) for bit in self.Alice.bit_string])
        self.Bob.bit_string[:] = np.array([int(bit) for bit in self.Bob.bit_string])
        block_order = channel_code.calculate_order(sum(self.sift_indices))

        # Creating the PARITY dictionary to be sent on a classical channel
        PARITY_DICT = channel_code.init_parity_dict(block_order)
        # Embedding the PARITY bits in alice's bit strings to form a block
        Alice_key_block, PARITY_DICT, num_fillers = channel_code.create_block_w_parity(self.Alice.bit_string, PARITY_DICT, encode = True)

        # Broadcasting on a classical channel
        BROADCAST = PARITY_DICT

        # Embedding the PARITY bits(received on the classical channel from alice) in bob's bit strings to form a block
        Bob_key_block, _, _ = channel_code.create_block_w_parity(self.Bob.bit_string, BROADCAST, encode = False)

        # reserve_i = int(self.Alice.current_length() - num_extras)
        # self.append_reserves(reserve_i)

        print(f'\t# {PARITY_DICT = }')
        print_info([Alice_key_block, Bob_key_block], '[Extended] Key BLOCKS(Alice-Bob Parity Encoded): ', True)
        print_info([self.Alice.bit_string, self.Bob.bit_string, self.Alice.bases, self.Bob.bases], 'Key strings(During Correction -- Not corrected): ', True)
        
        # Applying the hamming algorithm to: detect upto 2 bit errors and correct a single-bit error. 
        err_count, loc = channel_code.Apply_on(Bob_key_block, self.index)
        print(f" \t # {err_count = }, {loc = }, binary_rep = {bin(loc)[2:].zfill(block_order)}")
        
        if err_count != 0:
            try: 
                if err_count == 1: 
                    Bob_key_block[loc] = np.mod(Bob_key_block[loc] + 1, 2)
                    print(" \t - ERROR CORRECTED! Keeping the Key!")
                    self.status = 'keep'
    
                    if not Key.memory_efficient:
                        Key.kept = np.concatenate((Key.kept, [self.index]))

                else:
                    print(" \t - Discarding the Key!")
                    self.status = 'discard'

                    if not Key.memory_efficient:
                        Key.discarded = np.concatenate((Key.discarded, [self.index]))
                    
            except:
                raise KeyError('Location Invalid')

        for j, bit in enumerate(Bob_key_block):    # Or alice_bits, since both should be the same
            if j not in PARITY_DICT:
                self.Bob.key = np.concatenate((self.Bob.key, [bit]))

        # print(f'\t - Length of Sifted Key = {sum(self.sift_indices)}')
        self.Bob.key = self.Bob.key[:sum(self.sift_indices)]
        self.Bob.bit_string = self.Bob.key
        
        self.Alice.bit_string = np.array(self.Alice.bit_string, dtype='<U11')
        # self.Alice.bit_string = np.concatenate((self.Alice.bit_string, np.zeros(num_fillers, dtype='<U11')))

        self.remarks += f'\n {err_count} error(s) corrected'
### bits, bases = str, str

    def amplify_privacy(self):		
        #Generating seed (salt)
        seed = []
        for i in self.Alice.bit_string:
            a = randrange(2)
            seed.append(a)
        
        #Adding seeds to the keys
        self.Alice.bit_string = np.concatenate((self.Alice.bit_string, seed))
        self.Bob.bit_string = np.concatenate((self.Bob.bit_string, seed))
        
        #Converting lists to strings
        self.Alice.bit_string = ' '.join([str(elem) for elem in self.Alice.bit_string])
        self.Bob.bit_string = ' '.join([str(elem) for elem in self.Bob.bit_string])
        
        #checking first bit to decide hash function to use
        if self.Alice.bit_string[0] == 1:
            resultA = hashlib.sha256(self.Alice.bit_string.encode())
            self.Alice.amp_key = bin(int(resultA.hexdigest(), 16))[2:]
        else:
            resultA = hashlib.sha3_256(self.Alice.bit_string.encode())
            self.Alice.amp_key = bin(int(resultA.hexdigest(), 16))[2:]
        
        print()
        if self.Bob.bit_string[0] == 1:
            resultB = hashlib.sha256(self.Bob.bit_string.encode())
            self.Bob.amp_key = bin(int(resultB.hexdigest(), 16))[2:]
        else:
            resultB = hashlib.sha3_256(self.Bob.bit_string.encode())
            self.Bob.amp_key = bin(int(resultB.hexdigest(), 16))[2:]
            
    
    @classmethod
    def distribute(cls, Keys, num = None, size = None):

        if num == None: num = 16
        if size == None: size = 64
            
        # Accumulating all the keys and bases in their respective reservoirs
        for i in range(cls.num_keys):
            # Convert Alice's bit strings to strings and concatenate
            alice_bit_string = np.array(Keys[i].Alice.bit_string, dtype = '<U11')  # Adjust dtype to match
            cls.BitsReservoir[0] = np.concatenate((cls.BitsReservoir[0], alice_bit_string))
            
            bob_bit_string = np.array(Keys[i].Bob.bit_string, dtype = '<U11')  # Adjust dtype to match
            cls.BitsReservoir[1] = np.concatenate((cls.BitsReservoir[1], bob_bit_string))

            cls.BasesReservoir = np.concatenate((cls.BasesReservoir, Keys[i].Bob.bases))
            cls.Reservoir.update({i: (Keys[i].Bob.bases, Keys[i].Bob.bit_string, Keys[i].Alice.bit_string)})

        # Distributing equal length keys to each key
        cls.ReservoirLen = len(Key.BitsReservoir[0])
        
        key_len = int(size - np.log2(size) - 1)
        j = 0
        for i in range(0, num):
            # Assign a portion of the BitsReservoir to Alice and Bob
            Keys[i].Alice.bit_string = cls.BitsReservoir[0][j: j+key_len]
            Keys[i].Alice.bases = cls.BasesReservoir[j: j+key_len]
            Keys[i].Bob.bit_string = cls.BitsReservoir[1][j: j+key_len]
            Keys[i].Bob.bases = cls.BasesReservoir[j: j+key_len]
            j += key_len
            
        # Handling excess bits and bases
        cls.excess_Alice = AliceBob("excess_Alice")
        cls.excess_Bob = AliceBob("excess_Bob")
        
        cls.excess_Alice.bit_string = cls.BitsReservoir[0][j:]
        cls.excess_Alice.bases = cls.BasesReservoir[j:]
        cls.excess_Bob.bit_string = cls.BitsReservoir[1][j:]
        cls.excess_Bob.bases = cls.BasesReservoir[j:]
### bits, bases = str, str

#############

In [24]:
###############
def transfer_keys(snr_db, num_keys, key_length, signal_power, results, ECC='Cascade'):
    
    kept_i = np.zeros(num_keys, bool)
    discarded_i = np.zeros(num_keys, bool)
    Reservoir = {}
    # meta_qber  = np.zeros(num_keys, float)
    
    num_err_before = np.zeros(num_keys, int)    # Equivalent to meta qber -- after sifting; before correction
    num_err_after = np.zeros(num_keys, int)    # Equivalent to BER -- not accessible in realtime

    Keys = np.zeros(num_keys, object)
    for i in range(num_keys):

        start_key = time.time()

        Keys[i] = Key(i, key_length, snr_db)
        key_i = Keys[i]

        if i%10 == 0: 
            print(type(key_i))  # This should output <class 'Key'>
            print(hasattr(key_i, 'calculate_meta_qber'))  # This should return True

            key_i.generate_random_bits("Alice")
            key_i.generate_random_bases("Alice")
            print(key_i.Alice.bit_string)

            key_i.encode_states("Alice")
            key_i.NoisyChannel()            
            key_i.generate_random_bases("Bob")
            key_i.measure("Bob")
            key_i.sift()
            print_info([key_i.Alice.bit_string, key_i.Bob.bit_string, key_i.Alice.bases, key_i.Bob.bases], string = 'After Sifting|Before spotting', info = True)
            
            num_err_before[i] = np.sum(key_i.Alice.bit_string != key_i.Bob.bit_string)
            
            key_i.spot()
            print_info([key_i.Alice.bit_string, key_i.Bob.bit_string], string='After spotting | Before Cascade', info=True)
            key_i.calculate_meta_qber()

            ###########
            qber = key_i.qber
            alice_bits = key_i.Alice.bit_string
            bob_bits = key_i.Bob.bit_string
            print_info([alice_bits, bob_bits], string='Before Cascade(vars)', info = True)
            
            # Error correction based on QBER
            if qber > 0.25: 
                print('\n QBER > 0.25. Key Discarded')
                key_i.status = 'discard'
                discarded_i[i] = True
                continue

            bob_bits = cascade_biconf_error_correction(alice_bits, bob_bits, qber)
            bob_bits = to_int_list(bob_bits)
            key_i.status = 'keep'
            kept_i[i] = True

            num_err_after[i] = np.sum(key_i.Alice.bit_string != key_i.Bob.bit_string)
            
            try: print_info([alice_bits, bob_bits], string='After Cascade(vars)', info = True)
            except: 
                ('WARNING! Alice Bits and Bob Bits have different dimensions.')
                print(alice_bits, '\n', bob_bits)
                print(type(alice_bits), '\n', type(bob_bits))

            key_i.Alice.bit_string = alice_bits
            key_i.Bob.bit_string = bob_bits
            Reservoir.update({
                key_i.index: (key_i.Bob.bases, key_i.Bob.bit_string, key_i.Alice.bit_string)
            })

            # Privacy Amplification
            key_i.amplify_privacy()
            print(key_i.Alice.amp_key, key_i.Bob.amp_key)


        else: 
            key_i.generate_random_bits("Alice")
            key_i.generate_random_bases("Alice")
            key_i.encode_states("Alice")
            key_i.NoisyChannel()
            key_i.generate_random_bases("Bob")
            key_i.measure("Bob")
            key_i.sift()

            num_err_before[i] = np.sum(key_i.Alice.bit_string != key_i.Bob.bit_string)

            key_i.spot()
            key_i.calculate_meta_qber() # equivalent to num_err_before
            
            qber = key_i.qber
            alice_bits = key_i.Alice.bit_string
            bob_bits = key_i.Bob.bit_string
            
            print(f'{qber=}, {type(qber)=}')
            print(f"{results['avg_qber']=}, {type(results['avg_qber'])=}")
            # Error correction based on QBER
            if qber > 0.25: 
                key_i.status = 'discard'
                discarded_i[i] = True
                continue
                
            bob_bits = cascade_biconf_error_correction(alice_bits, bob_bits, qber)

            bob_bits = to_int_list(bob_bits)
            key_i.status = 'keep'
            kept_i[i] = True

            num_err_after[i] = np.sum(key_i.Alice.bit_string != key_i.Bob.bit_string)

            key_i.Alice.bit_string = alice_bits
            key_i.Bob.bit_string = bob_bits
            Reservoir.update({
                key_i.index: (key_i.Bob.bases, key_i.Bob.bit_string, key_i.Alice.bit_string)
            })

            # Privacy Amplification
            key_i.amplify_privacy()
        ## END IF-ELSE

        print(f'{qber=}, {type(qber)=}')
        print(f"{results['avg_qber']=}, {type(results['avg_qber'])=}")

        # Collecting data to average over number of keys; discarded key data won't be stored here
        results['avg_qber'] += qber
        results['avg_ber'] += key_i.ber
        results['avg_meta_qber'] += key_i.meta_qber
        results['avg_distilled_key_len'] += len(key_i.Bob.bit_string)
        results['avg_total_time'] += time.time() - start_key
               
        results['avg_corrected_qber'] += sum(key_i.qber for i in kept_i)/sum(kept_i) if sum(kept_i) > 0 else 0
        results['avg_corrected_ber'] += sum(key_i.ber for i in kept_i)/sum(kept_i) if sum(kept_i) > 0 else 0

    #     # Save a checkpoint after each key
    #     save_checkpoint({
    #         'qber': key_i.qber,
    #         'ber': key_i.ber,
    #         'meta_qber': key_i.meta_qber,   # It is not capturing data of discarded keys; this part is skipped by "continue"
    #         'corrected_qber': results['avg_corrected_qber'],
    #         'corrected_ber': results['avg_corrected_ber'],
    #         'distilled_key_len': len(key_i.Bob.bit_string)
    #     }, checkpoint_file)
    # ## END FOR

    # Individual Key level - not averaged; plotted
    # results['err_distribution_before'] = num_err_before     

    ### End For
    return Keys, kept_i, discarded_i, Reservoir

In [25]:
# Assuming 'transfer_keys' is defined elsewhere and imported

def simulate_snr_range(SNR_RANGE, num_keys, key_length, signal_power):
    # Initialize results for the entire SNR range
    avg_results = {
        'avg_qber': np.zeros(len(SNR_RANGE)),
        'avg_ber': np.zeros(len(SNR_RANGE)),
        'avg_corrected_qber': np.zeros(len(SNR_RANGE)),
        'avg_corrected_ber': np.zeros(len(SNR_RANGE)),
        'avg_meta_qber': np.zeros(len(SNR_RANGE)),
        'avg_distilled_key_len': np.zeros(len(SNR_RANGE)),
        'avg_total_time': np.zeros(len(SNR_RANGE)),
        'avg_skr': np.zeros(len(SNR_RANGE)),
        'avg_bps': np.zeros(len(SNR_RANGE)),
        'keys_kept': np.zeros(len(SNR_RANGE)),
        'keys_discarded': np.zeros(len(SNR_RANGE)),
        'total_kept_bits': np.zeros(len(SNR_RANGE)),
        # 'err_distribution_before': [None] * len(SNR_RANGE),  # For each SNR
        # 'err_distribution_after': [None] * len(SNR_RANGE)
    }

    for snr_index, snr_db in enumerate(SNR_RANGE):
        start_snr = time.time()
#         print(f'Starting simulation for SNR = {snr_db}')
        
        Key.error_prob = snr2ber(snr_db)  # Compute error probability for the SNR

        # Initialize storage for keys and errors for this SNR level
        Key.errors_present = np.zeros(num_keys, int)
        Key.errors_corrected = np.zeros(num_keys, int)
        # Keys = np.zeros(num_keys, dtype='object')
        
        # Per-SNR-level results
        results = {
            'avg_qber': 0.0,
            'avg_ber': 0.0,
            'avg_meta_qber': 0.0,
            'avg_corrected_qber': 0.0,
            'avg_corrected_ber': 0.0,
            'avg_distilled_key_len': 0.0,
            'avg_total_time': 0.0,
            # 'err_distribution_before': [],
            # 'err_distribution_after': []
        }

        # Call the 'transfer_keys' function to simulate key exchange and error correction
        Keys, kept_i, discarded_i, Reservoir = transfer_keys(snr_db, num_keys, key_length, signal_power, results)
        
        # Average over number of keys for this SNR
        for dict_key in results.keys():
            print(f"{dict_key}: {results[dict_key]}, type={type(results[dict_key])}")

            if isinstance(results[dict_key], list):  # Append the error distributions
                results[dict_key].append(results[dict_key])
            else:
                results[dict_key] = results[dict_key]/float(num_keys)  # Scalar values can be averaged

        # Aggregate Results after error correction
        total_kept_bits = np.sum([Keys[i].Bob.current_length() for i in range(num_keys) if kept_i[i]])
        keys_kept = np.sum(kept_i)
        time_taken = time.time() - start_snr
        
        avg_results['avg_skr'][snr_index] += keys_kept / time_taken if time_taken > 0 else 0
        avg_results['avg_bps'][snr_index] += total_kept_bits / time_taken if time_taken > 0 else 0
        avg_results['keys_discarded'][snr_index] += np.sum(discarded_i)
        avg_results['keys_kept'][snr_index] += keys_kept
        avg_results['total_kept_bits'][snr_index] += total_kept_bits

        for dict_key in results.keys():
            avg_results[dict_key][snr_index] += results[dict_key]  # Incrementally add results

        # Clean up large variables to avoid memory buildup
#         del Keys, results, Reservoir

    # Return aggregated results for this batch
    return avg_results

In [26]:
# Monte Carlo Simulation incorporating the multiprocessing
def monte_carlo(num_keys, key_length, sample_size, SNR_RANGE, signal_power, batch_size=None):

    start_time = time.time()

    # Initialize the dictionary to store the running averages
    avg_mcs_results = {
        'avg_qber': np.zeros(len(SNR_RANGE)),
        'avg_ber': np.zeros(len(SNR_RANGE)),
        'avg_corrected_qber': np.zeros(len(SNR_RANGE)),
        'avg_corrected_ber': np.zeros(len(SNR_RANGE)),
        'avg_meta_qber': np.zeros(len(SNR_RANGE)),
        'avg_distilled_key_len': np.zeros(len(SNR_RANGE)),
        'avg_total_time': np.zeros(len(SNR_RANGE)),
        
        'avg_skr': np.zeros(len(SNR_RANGE)),
        'avg_bps': np.zeros(len(SNR_RANGE)),
        'keys_kept': np.zeros(len(SNR_RANGE)),
        'keys_discarded': np.zeros(len(SNR_RANGE)),
        'total_kept_bits': np.zeros(len(SNR_RANGE)),
        
        # 'err_distribution_before': [None] * len(SNR_RANGE),  
        # 'err_distribution_after': [None] * len(SNR_RANGE)
    }

    # cores = cpu_count()  # Get the number of CPU cores
    # if batch_size is None:
    #     batch_size = cores

    # total_samples = 0  # Keep track of total samples processed
    # num_batches = int(np.ceil(sample_size / batch_size))
#     # Multiprocessing Pool
#     with mp.Pool(processes=cores) as pool:
#         # Process each batch
#         for batch_idx in range(num_batches):
#             current_batch_size = min(batch_size, sample_size - total_samples)

#             # Run simulations for the current batch in parallel using multiprocessing
#             parallel_results = pool.starmap(simulate_snr_range, [(SNR_RANGE, num_keys, key_length, signal_power)
#                                                                  for _ in range(current_batch_size)])

#             # Update running averages after each batch
#             for result in parallel_results:
#                 for key in avg_mcs_results.keys():
# #                     if isinstance(avg_mcs_results[key], list):
# #                         # Handle lists (append results)
# #                         for idx, values in enumerate(result[key]):
# # #                             if avg_mcs_results[key][idx] is None:
# # #                                 avg_mcs_results[key][idx] = values
# # #                             else:
# #                                  avg_mcs_results[key][idx] += values
# #                     else:
#                     avg_mcs_results[key] = (avg_mcs_results[key] * total_samples + result[key] * current_batch_size) / (total_samples + current_batch_size)

#             total_samples += current_batch_size
            
#             track(batch_idx, num_batches)

    for sample in range (sample_size):
        results = simulate_snr_range(SNR_RANGE, num_keys, key_length, signal_power)
        for key in avg_mcs_results.keys():
            avg_mcs_results[key] += results[key]
        track(sample, sample_size)
    
    for key in avg_mcs_results.keys():
        avg_mcs_results[key] = avg_mcs_results[key]/sample_size

    end_time = time.time()
    print(f"Monte Carlo simulation completed in {end_time - start_time:.2f} seconds.")

    return avg_mcs_results

In [27]:
%%time
# Example Simulation Parameters
sample_size = int(1e2)  # 100,000 samples
desired_key_len = 32
num_keys = int(1e2)  # 1000 keys
SNR_RANGE = np.arange(0, 12.5, 0.5)
signal_power = 0.15

key_length = 3*desired_key_len
memory_efficient = True

Key.num_keys = num_keys
# Run the Monte Carlo Simulation with multiprocessing and optional acceleration
results = monte_carlo(num_keys, key_length, sample_size, SNR_RANGE, signal_power)

<class '__main__.Key'>
True
['0' '1' '1' '1' '0' '0' '1' '0' '1' '0' '1' '0' '0' '1' '1' '0' '0' '0'
 '1' '0' '0' '0' '1' '0' '0' '1' '0' '0' '0' '1' '0' '0' '1' '1' '1' '0'
 '0' '1' '1' '1' '1' '1' '0' '1' '0' '1' '1' '0' '0' '1' '0' '1' '0' '0'
 '1' '1' '0' '1' '1' '1' '0' '1' '1' '1' '1' '1' '0' '0' '0' '0' '0' '1'
 '0' '1' '1' '1' '1' '0' '0' '1' '1' '1' '1' '0' '1' '1' '1' '1' '1' '1'
 '0' '0' '0' '0' '1' '0']

######## After Sifting|Before spotting			 ----- Number of Variables: 4, 	 Number of Elements: 57
-> Element Type: <class 'numpy.str_'>
-> Variable Types: <class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>, <class 'numpy.ndarray'>	 ---> Variable Shapes: (57,), (57,), (57,), (57,)

01ZZ <    11XX      00XX      10ZZ <    00ZZ      00ZZ      11ZZ      00ZZ      11XX      00XX      00ZZ      00XX      10ZZ <    01ZZ <    01ZZ <    10XX <    00XX      00ZZ      10ZZ <    00ZZ      00XX      11ZZ      01ZZ <    10XX <    00ZZ      01XX <    11XX      11XX  

KeyboardInterrupt: 

In [28]:

results

NameError: name 'results' is not defined